# 1. Import / Clean Dataset

In [29]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import numpy as np
from scipy.optimize import minimize # for loss func minimization
from collections import defaultdict
from sklearn.metrics import classification_report, confusion_matrix
from copy import deepcopy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt # for plotting stuff
from sklearn.metrics import log_loss
from scipy import optimize
import copy
import itertools
from random import seed, shuffle
import math
from sklearn.svm import SVC
from sklearn.utils import shuffle


In [30]:
from google.colab import drive
drive.mount('/content/drive')
import os
from glob import glob
os.getcwd()
glob('/content/drive/MyDrive/*')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['/content/drive/MyDrive/compas-scores-two-years.csv',
 '/content/drive/MyDrive/Colab Notebooks',
 '/content/drive/MyDrive/A1 .ipynb',
 '/content/drive/MyDrive/main.ipynb']

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/compas-scores-two-years.csv')
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [ ]:
# Feature selection: we add jail time
df['jail_time']=(df['c_jail_out'].apply(pd.to_datetime) - df['c_jail_in'].apply(pd.to_datetime)).dt.days

In [ ]:
feauture_name = ['age', "race", "sex", "priors_count", "c_charge_degree","jail_time","decile_score", 'score_text', 'two_year_recid']

df_new = df[feauture_name]
df_clean = df_new[(df['race'] == 'African-American') | (df['race'] == 'Caucasian')]
df_clean.head()


,age,race,sex,priors_count,c_charge_degree,jail_time,decile_score,score_text,two_year_recid
1,34,African-American,Male,0,F,10.0,3,Low,1
2,24,African-American,Male,4,F,1.0,4,Low,1
3,23,African-American,Male,1,F,NaN,8,High,0
6,41,Caucasian,Male,14,F,6.0,6,Medium,1
8,39,Caucasian,Female,0,M,2.0,1,Low,0


In [ ]:
df_clean.dropna(inplace = True)

In [ ]:
df_clean.head()

,age,race,sex,priors_count,c_charge_degree,jail_time,decile_score,score_text,two_year_recid
1,34,African-American,Male,0,F,10.0,3,Low,1
2,24,African-American,Male,4,F,1.0,4,Low,1
6,41,Caucasian,Male,14,F,6.0,6,Medium,1
8,39,Caucasian,Female,0,M,2.0,1,Low,0
9,21,Caucasian,Male,1,F,0.0,3,Low,1


In [ ]:
X = df_clean[['age', "race", "sex", "priors_count", "c_charge_degree","jail_time","decile_score", 'score_text']]
y = df_clean['two_year_recid']

## Algorithm 1: Fairness Constraints: Mechanisms for Fair Classification (Yanzhao & Lei)

In [ ]:
data = df[(df['race']=='African-American')|(df['race']=='Caucasian')]
data["length_of_stay"] = (pd.to_datetime(data.c_jail_out) - pd.to_datetime(data.c_jail_in)).dt.total_seconds() / (24 * 3600)
data = data[(data.days_b_screening_arrest <= 30) & (data.days_b_screening_arrest >= -30)]
data = data[data.is_recid != -1]
data = data[data.c_charge_degree != "O"]
data = data[data.score_text != "N/A"]
data.drop(['first','last','c_case_number','c_charge_desc','violent_recid','vr_charge_degree','vr_case_number','vr_offense_date','vr_charge_desc',
          'c_arrest_date','r_jail_out','r_jail_in','r_days_from_arrest','r_charge_desc',
          'r_offense_date','r_case_number','r_charge_degree'], axis=1,inplace=True)
data['c_days_from_compas'] = data['c_days_from_compas'] .fillna(data['c_days_from_compas'].mode()[0])
data = data.dropna()
X = data.drop(['id','two_year_recid','dob','name','v_type_of_assessment','type_of_assessment'], axis=1)
X.set_index(data.id,inplace=True)
y = data[['two_year_recid']]
y.set_index(data.id,inplace=True)
X = X.drop(['c_offense_date','c_jail_in','c_jail_out','out_custody','in_custody','screening_date','compas_screening_date','v_screening_date'],axis=1)
#convert categorical variables into numeric
X['race']= X['race'].replace(['African-American','Caucasian'],[0,1.0])
X['sex'] = X['sex'].replace(['Male','Female'],[0,1.0])
X['age_cat'] = X['age_cat'].replace(['25 - 45','Less than 25', 'Greater than 45'],[0,1.0,2.0])
X['c_charge_degree'] = X['c_charge_degree'].replace(['M','F'],[0,1.0])
X['score_text'] = X['score_text'].replace(['Low', 'High', 'Medium'],[0,1.0,2.0])
X['v_score_text'] = X['v_score_text'].replace(['Low', 'High', 'Medium'],[0,1.0,2.0])
X = X.fillna(0)


y=np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1,random_state=44)
x_sensitive_tr = X_train.loc[:, 'race']
x_sensitive_te = X_test.loc[:, 'race']
X_train = X_train.loc[:, X_train.columns != 'race']
X_test = X_test.loc[:, X_test.columns != 'race']

X_train=X_train.reset_index()
X_test=X_test.reset_index()
x_sensitive_tr=x_sensitive_tr.reset_index()
x_sensitive_te=x_sensitive_te.reset_index()

X_train=X_train.drop(['id'], axis=1)
X_test=X_test.drop(['id'], axis=1)
x_sensitive_tr=x_sensitive_tr.drop(['id'], axis=1)
x_sensitive_te=x_sensitive_te.drop(['id'], axis=1)

In [39]:
model_base = LogisticRegression(fit_intercept=False).fit(X_train,y_train.flatten())
base_y_train = model_base.predict(X_train)
base_y_test = model_base.predict(X_test)

theta_star = model_base.coef_.flatten()

#metrics
print('train accuracy: ' + str(model_base.score(X_train,y_train.flatten())))
print('test accuracy: ' + str(model_base.score(X_test,y_test.flatten())))

# Assuming X_train, y_train, x_sensitive_tr['race'] are available as pandas DataFrame/Series

def calculate_group_calibration(predictions, actuals, sensitive_attribute):
    # Combine predictions, actuals, and sensitive attributes into a single DataFrame
    data = pd.DataFrame({
        'predictions': predictions,
        'actuals': actuals,
        'group': sensitive_attribute
    })

    # Group by the sensitive attribute and calculate means
    group_stats = data.groupby('group').agg({
        'predictions': 'mean',
        'actuals': 'mean'
    })

    # Calculate the calibration measure for each group
    group_stats['calibration'] = np.abs(group_stats['predictions'] - group_stats['actuals'])

    return group_stats['calibration'].mean()

# Predict probabilities if your model supports it
prob_y_train = model_base.predict_proba(X_train)[:, 1]  # Assuming binary classification
prob_y_test = model_base.predict_proba(X_test)[:, 1]

# Calculate calibration without using ut
train_calibration = calculate_group_calibration(prob_y_train, y_train.flatten(), x_sensitive_tr['race'].values)
test_calibration = calculate_group_calibration(prob_y_test, y_test.flatten(), x_sensitive_te['race'].values)

print('train calibration:', train_calibration)
print('test calibration:', test_calibration)

train accuracy: 0.9617433414043584
test accuracy: 0.9607843137254902
train calibration: 0.0061701928516699756
test calibration: 0.006230752822441593


In [43]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Assuming necessary functions are defined as adjust_model_coefficients and calculate_group_calibration
# Ensure you have defined or imported these functions here before using them
import numpy as np
import pandas as pd

def adjust_model_coefficients(X, y, sensitive_attribute, base_coefficients, g):
    # Convert to DataFrame for easier manipulation
    data = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    data['outcome'] = y
    data['group'] = sensitive_attribute

    # Overall positive outcome ratio
    overall_pos_ratio = np.mean(y)

    # Adjust coefficients based on group
    adjusted_coefficients = np.array(base_coefficients, dtype=float)  # Ensure we're working with a mutable float array
    groups = data['group'].unique()

    for group in groups:
        group_data = data[data['group'] == group]
        group_pos_ratio = np.mean(group_data['outcome'])

        # Scale factor based on the group's outcome ratio compared to overall
        scale_factor = group_pos_ratio / overall_pos_ratio

        # Adjust the coefficients for this group (simplistic approach)
        # g could be used to modulate the strength of the adjustment
        adjusted_coefficients *= (1 + g * (scale_factor - 1))

    return adjusted_coefficients

for g in [0, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2]:
    print('Using gamma = ', g)

    # Adjust the coefficients based on gamma value and group
    theta_g = adjust_model_coefficients(X_train, y_train.flatten(), x_sensitive_tr['race'].values, theta_star, g)

    # Initialize and set the logistic regression model with adjusted coefficients
    model_gamma = LogisticRegression(fit_intercept=False)
    model_gamma.coef_ = theta_g.reshape((1, -1))
    model_gamma.intercept_ = 0
    model_gamma.classes_ = np.array([0, 1])

    # Predict and calculate metrics
    g_y_train = model_gamma.predict(X_train)
    g_y_test = model_gamma.predict(X_test)

    # Metrics
    print('train accuracy: ' + str(model_gamma.score(X_train, y_train.flatten())))
    print('test accuracy: ' + str(model_gamma.score(X_test, y_test.flatten())))

    # Calculate calibration manually
    prob_y_train = model_gamma.predict_proba(X_train)[:, 1]
    prob_y_test = model_gamma.predict_proba(X_test)[:, 1]

    g_calib_train = calculate_group_calibration(prob_y_train, y_train.flatten(), x_sensitive_tr['race'].values)
    g_calib_test = calculate_group_calibration(prob_y_test, y_test.flatten(), x_sensitive_te['race'].values)

    print('train calibration difference: ' + str(g_calib_train))
    print('test calibration difference: ' + str(g_calib_test))


Using gamma =  0
train accuracy: 0.9617433414043584
test accuracy: 0.9607843137254902
train calibration difference: 0.0061701928516699756
test calibration difference: 0.006230752822441593
Using gamma =  0.001
train accuracy: 0.9617433414043584
test accuracy: 0.9607843137254902
train calibration difference: 0.006170016967788766
test calibration difference: 0.006230638018174667
Using gamma =  0.01
train accuracy: 0.9617433414043584
test accuracy: 0.9607843137254902
train calibration difference: 0.006168426539309435
test calibration difference: 0.006229601598383427
Using gamma =  0.05
train accuracy: 0.9617433414043584
test accuracy: 0.9607843137254902
train calibration difference: 0.006161193855556801
test calibration difference: 0.006224926722863733
Using gamma =  0.1
train accuracy: 0.9617433414043584
test accuracy: 0.9607843137254902
train calibration difference: 0.006151769876254931
test calibration difference: 0.006218929089675945
Using gamma =  0.15
train accuracy: 0.96174334140435

In [44]:
def adjust_coefficients_fairness(X, y, sensitive_attribute, base_coefficients, gamma):
    data = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    data['outcome'] = y
    data['group'] = sensitive_attribute

    # Calculate the mean outcome for each group
    group_means = data.groupby('group')['outcome'].mean().to_dict()

    # Overall mean outcome
    overall_mean = y.mean()

    # Adjust coefficients
    adjusted_coeffs = np.array(base_coefficients)
    for group, mean in group_means.items():
        adjustment_factor = gamma * (mean - overall_mean)
        adjusted_coeffs += adjustment_factor  # Adjust based on the group mean deviation and gamma

    return adjusted_coeffs
theta_star = model_base.coef_.flatten()  # Get initial model coefficients

for g in [0.1]:
    print('Using gamma = ', g)

    # Adjust coefficients considering fairness
    theta_fg = adjust_coefficients_fairness(X_train.values, y_train.flatten(), x_sensitive_tr['race'].values, theta_star, g)

    # Set up the model with adjusted coefficients
    model_fine_gamma = LogisticRegression(fit_intercept=False)
    model_fine_gamma.coef_ = theta_fg.reshape((1, -1))
    model_fine_gamma.intercept_ = 0
    model_fine_gamma.classes_ = np.array([0, 1])

    # Predict and score
    fg_y_train = model_fine_gamma.predict(X_train)
    fg_y_test = model_fine_gamma.predict(X_test)

    # Print accuracy
    print('train accuracy: ' + str(model_fine_gamma.score(X_train, y_train.flatten())))
    print('test accuracy: ' + str(model_fine_gamma.score(X_test, y_test.flatten())))

    # Calculate calibration manually
    prob_fg_y_train = model_fine_gamma.predict_proba(X_train)[:, 1]
    prob_fg_y_test = model_fine_gamma.predict_proba(X_test)[:, 1]
    fg_calib_train = calculate_group_calibration(prob_fg_y_train, y_train.flatten(), x_sensitive_tr['race'].values)
    fg_calib_test = calculate_group_calibration(prob_fg_y_test, y_test.flatten(), x_sensitive_te['race'].values)

    print('train calibration difference: ' + str(fg_calib_train))
    print('test calibration difference: ' + str(fg_calib_test))


Using gamma =  0.1
train accuracy: 0.9508474576271186
test accuracy: 0.9586056644880174
train calibration difference: 0.033152779982352165
test calibration difference: 0.03212959043454783
